In [2]:
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers, models
import tensorflow_hub as hub
import pathlib
import os

In [3]:
import deeplake
import numpy as np

In [ ]:
# Loads plant_village dataset
ds = deeplake.load('hub://activeloop/plantvillage-without-augmentation')

In [ ]:
# Prints shape of plant_village images after turning it into np.ndarray format
# This cell takes five minutes to run
img = ds.images.numpy()
print(np.shape(img))
print(type(img))

In [ ]:
img.shape

In [ ]:
# Prints shape of plant_village labels
label = ds.labels.numpy(aslist=True)
labels = np.array(label)

print(labels.shape)
print(labels[:10])

In [ ]:
# Clones PlantDoc dataset from GitHub into Google Colab
# This cell could take a few minutes
# print(os.getcwd())
!git clone https://github.com/pratikkayal/PlantDoc-Dataset.git

In [ ]:
# Access PlantDoc training dataset
# print(os.getcwd())
data_dir = pathlib.Path("PlantDoc-Dataset/train").with_suffix("")

image_count_train = len(list(data_dir.glob('*/*.jpg')))

batch_size = 32
img_height = 256
img_width = 256

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

# class_names = train_ds.class_names
# print(class_names)

# Access PlantDoc testing dataset
data_dir = pathlib.Path("PlantDoc-Dataset/test").with_suffix("")
image_count_test = len(list(data_dir.glob('*/*.jpg')))

test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
# Convert data into np.ndarray format

plantDoc_images = np.ndarray([1, 256, 256, 3])
plantDoc_labels = np.ndarray([1])
plantDoc_images_test = np.ndarray([1, 256, 256, 3])
plantDoc_labels_test = np.ndarray([1])

count = 0
for image_batch, labels_batch in train_ds:
#   if count % 5 == 0:
#     print("Batch number", count)
  plantDoc_images = np.append(plantDoc_images, image_batch.numpy(), axis = 0)
  plantDoc_labels = np.append(plantDoc_labels, labels_batch.numpy(), axis = 0)
  count += 1

print(plantDoc_images.shape, plantDoc_labels.shape)

count = 0
for image_batch, labels_batch in test_ds:
#   if count % 5 == 0:
#     print("Batch number", count)
  plantDoc_images_test = np.append(plantDoc_images_test, image_batch.numpy(), axis = 0)
  plantDoc_labels_test = np.append(plantDoc_labels_test, labels_batch.numpy(), axis = 0)
  count += 1

print(plantDoc_images_test.shape, plantDoc_labels_test.shape)

In [ ]:
# Shuffle data
np.random.shuffle(img)
np.random.shuffle(labels)

np.random.shuffle(plantDoc_images)
np.random.shuffle(plantDoc_labels)
np.random.shuffle(plantDoc_images_test)
np.random.shuffle(plantDoc_labels_test)

In [ ]:
# Concatenate datasets
train_images = np.append(img[0:40727], plantDoc_images, axis = 0)
test_images = np.append(img[40728:52803], plantDoc_images_test, axis = 0)

train_labels = np.append(labels[0:40727], plantDoc_labels, axis = 0)
test_labels = np.append(labels[40728:52803], plantDoc_labels_test, axis = 0)

In [ ]:
# Save data to files
with open("train_images.pkl", mode="wb") as opened_file:
    pickle.dump(train_images, opened_file)
with open("test_images.pkl", mode="wb") as opened_file:
    pickle.dump(test_images, opened_file)
with open("train_labels.pkl", mode="wb") as opened_file:
    pickle.dump(train_labels, opened_file)
with open("test_labels.pkl", mode="wb") as opened_file:
    pickle.dump(test_labels, opened_file)

In [6]:
#pretrain model from tensorflow hub

url = "https://tfhub.dev/tensorflow/faster_rcnn/resnet152_v1_1024x1024/1"

base_model = hub.KerasLayer(handle=url, input_shape=(256, 256, 3))
base_model.trainable = False
# model = keras.Sequential([
#     base_model,
#     layers.Dense(38, activation="relu"),
# ])
print(type(base_model))
model = keras.Sequential()
model.add(base_model)
model.add(layers.Dense(38, activation="relu"))

Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x0000023D31849BC0>
Traceback (most recent call last):
  File "c:\Users\mail2\anaconda3\envs\bwsi_final\Lib\weakref.py", line 370, in remove
    self = selfref()
           ^^^^^^^^^
KeyboardInterrupt: 


<class 'tensorflow_hub.keras_layer.KerasLayer'>


TypeError: Exception encountered when calling layer "keras_layer_2" (type KerasLayer).

in user code:

    File "c:\Users\mail2\anaconda3\envs\bwsi_final\Lib\site-packages\tensorflow_hub\keras_layer.py", line 234, in call  *
        result = f()

    TypeError: Binding inputs to tf.function `f` failed due to `Tensor conversion requested dtype uint8 for Tensor with dtype float32: <tf.Tensor 'Placeholder:0' shape=(None, 256, 256, 3) dtype=float32>`. Received args: (<tf.Tensor 'Placeholder:0' shape=(None, 256, 256, 3) dtype=float32>,) and kwargs: {} for signature: (input_tensor: TensorSpec(shape=(1, None, None, 3), dtype=tf.uint8, name=None)).


Call arguments received by layer "keras_layer_2" (type KerasLayer):
  • inputs=tf.Tensor(shape=(None, 256, 256, 3), dtype=float32)
  • training=None

In [7]:
#trying using the resnet-152 model from keras

model = keras.applications.ResNet152(include_top=True)
base_inputs = model.layers[0].input
base_outputs = model.layers[-2].output
final_outputs = layers.Dense(38)(base_outputs)

tuned_model = keras.Model(inputs=base_inputs, outputs=final_outputs)
print(tuned_model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                          

In [8]:
tuned_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

In [ ]:
small_train = img[0:10]
small_labels = labels[]

In [ ]:
tuned_model.fit()